# Model Training

This notebook is a continuation of the model_selection page in which we attempted to train several prepackaged Scikit-Learn models on our feature data. None of the models we found were much more effective than random chance at predicting our features, if they were better at all. In the end we concluded that our system must be non-linear, thus in this noetbook we will be using Keras to build a deep learning solution to this system.

### Linear vs Non-Linear Optimization

All machine learning can be considered mathematical optimization. Even neural networks, though often spoken of as "simulations" of human brains, can only really be considered homages to organic brains. In our last notebook we were treating our data as a linear system, which would make machine learning (relatively) simple as each input has a single, clear output for the algorithm to attempt to estimate. It would have been very nice if this was the case, as a nonlinear system could have results that are not linearly seperable and are therefore not able to be solved with the same methods.

### Deep Learning

Deep neural networks have been used to model nonlinear systems and relationships in the past. They are particularly useful in image recognition, natural language processing, and more, and since we are using the Mel-spectrum to represent our music data we may be able to use the same systems here. Ultimately, a DNN is very similar to an ANN, with the exception that the DNN has many hidden layers between the input and output layers. Historically, convolutional neural networks have been used to solve genre classification problem in both single and multi-label contexts, so we will be exploring them here in combination with recurrent neural networks which are able to analyze time sequence data.

### Keras

Keras is a python API for building deep learning machine learning models with strong support for both convolutional and recurrent neural networks. It was built on top of the popular Theano library and effectively acts as a wrapper for it, however newer releases also allow for integration with Google's TensorFlow library instead. We will be using it to build our DNN below. It can be run on either CPU or GPU, we will be using the CPU option due to technical and budget limitations.

### Creation of DLGINN, the Deep Learning Genre Identification Neural Network

In [27]:
import numpy as np
import mysql.connector as dbc
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, Activation, Flatten, Dense

We need to re import all of our data, so we will reuse the code from model_selection (see model_selection.ipynb for more information on the cell below)

In [5]:
db = dbc.connect(port=3306,
                 user="root",
                 passwd="password",
                 db="SONG")
cursor = db.cursor()

query = "select * from SONG.FEATURES natural join SONG.GENRES"
cursor.execute(query)

data = []

# Data is far too big to call fetchall()
for row in cursor: 
    data.append(row)

data = np.array(data) 

X = data[:,1:11]
y = data[:,11:27]

In [6]:
frame = DataFrame(X)
frame.head()

,0,1,2,3,4,5,6,7,8,9
0,-515.8460,1.76513,-6.10513,0.538926,-3.27525,-2.62195,-2.706050,-3.74004,0.505801,2.55935
1,-60.3773,194.72600,-68.84740,49.278700,13.29830,1.62496,-0.189355,16.36540,-7.216030,-0.07875
2,-53.7426,193.48200,-81.19240,46.309600,20.14740,19.24740,-3.892020,11.21450,-7.568530,3.30256
3,-50.4798,198.53400,-79.67700,42.577300,14.63660,7.98548,-6.826490,10.38210,-6.121840,6.21249
4,-82.8048,183.97800,-75.45900,53.186100,20.62800,3.93690,-6.771320,9.39080,0.048497,7.43601


In [7]:
frame = DataFrame(y)
frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=73)
input_shape = (None, X_train.shape[1])

Now we can begin building our model, piece by piece, using the Keras Sequential object.

In [36]:
model = Sequential()


model.add(Conv1D(6, 5, input_shape=input_shape))
model.add(Activation('tanh'))

model.add(Conv1D(10, 5))
model.add(Activation('tanh'))

model.add(Conv1D(16, 5))
model.add(Activation('tanh'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, None, 6)           306       
_________________________________________________________________
activation_27 (Activation)   (None, None, 6)           0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, None, 10)          310       
_________________________________________________________________
activation_28 (Activation)   (None, None, 10)          0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, None, 16)          816       
_________________________________________________________________
activation_29 (Activation)   (None, None, 16)          0         
Total params: 1,432
Trainable params: 1,432
Non-trainable params: 0
_________________________________________________________________


### Evaluation

With a model in hand we can now train and evaluate it on our data. We will be using Scikit-Learn's train_test_split to make this a bit easier on ourselves.

In [38]:
model.fit(X_train, y_train)

score_train = model.evaluate(X_train, y_train)
score_test = model.evaluate(X_test, y_test)

print("Training Data Accuracy: {}".format(score_train[1]))
print("Test Data Accuracy: {}".format(score_test[1]))

ValueError: Error when checking input: expected conv1d_29_input to have 3 dimensions, but got array with shape (858484, 10)